### Get TikTok data

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from datetime import timedelta

In [2]:
today = (dt.datetime.today() + timedelta(days=30)).strftime('%Y-%m-%d')
today

'2022-02-15'

In [3]:
payload={}
headers = {
  'X-API-KEY': 'QT8K1SQ333HurjoXkTVKOchKswsI83cS'
}

In [4]:
count = '30'
secUid = 'MS4wLjABAAAAn982fxeWVi0MRU_Q0Lg0mVJ9AmAZ-GvWgjy9rSGReFPs_xkQSFkKoUnkuWTVhryi'

In [5]:
rng = pd.date_range(start='2021-03-01', end = today, freq='M')
df = pd.DataFrame({ 'date' : rng }, )

In [6]:
df['cursor'] = (pd.to_datetime(df['date']) - dt.datetime(1970,1,1)).dt.total_seconds().round().astype(str).str.replace('.0', '', regex=False) + '000'

In [7]:
cursors = df['cursor'].to_list()

In [8]:
datas = []

for c in cursors:
    url = f"https://api.tikapi.io/public/posts?count={count}&secUid={secUid}&cursor={c}"
    r = requests.request("GET", url, headers=headers, data=payload)
    datas.append(r.json())

In [9]:
len(datas)

11

In [10]:
def simple_dict(tiktok_dict):
    to_return = {}
    to_return['user_name'] = d['author']['uniqueId']
    to_return['video_id'] = d['id']
    to_return['video_desc'] = d['desc']
    to_return['video_time'] = d['createTime']
    to_return['video_duration'] = d['video']['duration']
    to_return['n_shares'] = d['stats']['shareCount']
    to_return['n_likes'] = d['stats']['diggCount']
    to_return['n_comments'] = d['stats']['commentCount']
    to_return['n_plays'] = d['stats']['playCount']
    to_return['video_link'] = 'https://www.tiktok.com/@{}/video/{}?lang=en'.format(to_return['user_name'], to_return['video_id'])
    return to_return

In [11]:
dataframe = []

for data in datas:
    for d in data['itemList']:
        dataframe.append(simple_dict(d))

In [12]:
df = pd.DataFrame(dataframe)

In [13]:
df = df.sort_values('video_time', ascending=False).drop_duplicates(subset='video_id').reset_index(drop=True)

In [14]:
df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link
0,jongraz,7053837902991658287,his message today is…CLEAR 🦴🔮🦴,1642349620,59,2131,46300,1150,181900,https://www.tiktok.com/@jongraz/video/70538379...
1,jongraz,7053542258251124014,Reply to @psxiii our official stance,1642280785,59,57,17200,834,141200,https://www.tiktok.com/@jongraz/video/70535422...
2,jongraz,7053460263864962350,Reply to @22hgarcia layers alert 🚨,1642261694,14,3767,179500,1512,891600,https://www.tiktok.com/@jongraz/video/70534602...
3,jongraz,7053454719641029934,you heard him! 🔮🦴🔮,1642260404,50,5643,142600,2337,523400,https://www.tiktok.com/@jongraz/video/70534547...
4,jongraz,7053079129113021742,thank you for all the nominations!! we’ll do t...,1642172956,57,202,27500,400,189200,https://www.tiktok.com/@jongraz/video/70530791...


In [15]:
len(df)

167

In [16]:
df.to_csv('data/processed/jongraz_videos.csv',index=False)